# Дз 3. Тестирование статистических гипотез

<div class="alert alert-info" role="alert">

Важно!

* Плагиат карается обнулением работы
* При использовании чужого кода из открытых источников необходимо указать ссылку на источник
* В каждой задаче необходимо обосновывать выбор статистического критерия, формулировать нулевую и альтернативную гипотезу и делать интерпретацию получившихся результатов, а также содержательный вывод.
* Весь текст оформляется строго в ячейках типа markdown

## Задача 1. Осьминог Пауль (2 балла)


В период с 2008 по 2010 год в Германии жил Осьминог Пауль, который был известен тем, что мог предсказывать результаты матчей с участием сборной Германии.     
   
В таблице ниже вы можете видеть результаты его предсказаний: 


<a href="https://ibb.co/VWnc5yG"><img src="https://i.ibb.co/QFS3tBV/image.jpg" alt="image" border="0"></a>

Если считать, что случайным образом мы угадывали ли бы 50% матчей, то можно ли утверждать, что осьминог Пауль был талантливым экстрасенсом со способностями к предсказаниям? Важно! Учитывайте только факт угадывания или неугадывания, остальные параметры матчей не важны.

In [312]:
import numpy as np
from scipy import stats

H0: Пауль угадывает с вероятность p = 0.5.
H1: p > 0.5

In [313]:
data = [1,0,1,1,1,0,1,1,1,1,1,1,1]
n = len(data)
sigma = np.std(data, ddof=1)
mean = np.mean(data)
p_h0 = 0.5
alha = 0.05
mean, p_h0

(0.8461538461538461, 0.5)

In [314]:
t_stat = (mean-p_h0)/(sigma/np.sqrt(n))

In [315]:
p_value = 1 - stats.t.cdf(t_stat, n-1)
t_stat, p_value

(3.323470256441984, 0.003035524374994991)

In [316]:
t_stat1, p_value1 = stats.ttest_1samp(data, p_h0, alternative='greater')
t_stat1, p_value

(3.323470256441984, 0.003035524374994991)

Отвергаем H0 на любом разумном уровне значимости, Пауль умеет предсказывать исход матчей.

## Задача 2.  Корм и котики (2 балла)

    
Новый корм для котиков рекламируется с помощью рекламного баннера. Отдел маркетинга разработал для рекламы новый баннер. Они считают, что он более красивый. Перед фирмой возникла необходимость проверить какой из двух баннеров лучше.

Для этого была создана специальная веб-форма. В этой веб-форме людям по очереди демонстрировали оба баннера и просили поставить лайк, если баннер нравится человеку. Если человеку нравятся оба баннера, он может поставить два лайка. Если ему не нравится ни один баннер, он может не ставить лайки вовсе. Собранные в конечном итоге данные находятся в табличке.

Можно ли утверждать, что оба баннера одинаково хороши? Сформулируйте статистическую гипотезу для проверки того, что оба баннера выбирают одинаково часто. Проверьте ее и сделайте выводы.


In [317]:
import pandas as pd

H0: банеры выбирают одинаково часто. H1: второй баннер людям нравится больше

In [318]:
df = pd.read_csv('banner_click_stat.csv', sep = '\t', names=['first_banner', 'second_banner'])
df.head()

,first_banner,second_banner
0,0,0
1,1,1
2,0,0
3,0,0
4,0,0


In [319]:
sample_first_banner = df['first_banner']
sample_second_banner = df['second_banner']

In [320]:
mean_first = np.mean(sample_first_banner)
mean_second = np.mean(sample_second_banner)
n1 = sample_first_banner.shape[0]
n2 = sample_second_banner.shape[0]
var1 = np.var(sample_first_banner, ddof=1)
var2 = np.var(sample_second_banner, ddof=1)
sigma = np.sqrt(var1/n1 + var2/n2)
mean_first, mean_second, sigma

(0.037, 0.053, 0.009268651892692227)

In [329]:
ztest = (mean_first-mean_second)/sigma
p_value = stats.norm.cdf(ztest)
print("ztest:", ztest, "p_value:", p_value)

ztest: -1.7262488855164617 p_value: 0.042151323371125725


In [330]:
ttest = (mean_first-mean_second)/sigma
p_value = stats.t.cdf(ttest, n1-2)
print("ttest:", ttest, "p_value:", p_value)

ttest: -1.7262488855164617 p_value: 0.042306066331333964


In [326]:
ttest1, pval = stats.ttest_ind(sample_first_banner, sample_second_banner, alternative='less')
print("ttest1:", ttest1, "p_value", pval)

ttest1: -1.7262488855164546 p_value 0.04222861771128191


H0 отвергается на уровне значимости 0.05, втрой баннер лучше

## Задача 3. Немного АВ-тестов (3 балла)
## Игра Cookie Cat

В этом задании мы поработаем с данными Cookie Cats - популярной игры-головоломки, которую разработала компания Tactile Entertainment. В этой игре игрок должен соединять одинаковые элементы в ряд, чтобы очистить поле и выиграть уровень. 

<p style="align: center;"><img align=center src="https://i.ytimg.com/vi/iPxZIp0cbJE/maxresdefault.jpg"  width=500></p>
 
 
<p style="align: center;"><img align=center src="https://s3.amazonaws.com/assets.datacamp.com/production/project_184/img/cc_gates.png"  width=500></p>


По мере прохождения уровней перед игроками появляются ворота, открытия которых нужно ждать какое-то время или пропускать их, совершая покупки в приложении. С точки зрения создателей эти ворота повышают интерес к игре и стимулируют к вложению денег в игру.


Но у создателей появляется важный вопрос: **где лучше всего ставить ворота?**
Первоначально первые ворота были размещены на уровне 30. Однако есть гипотеза, что лучше переместить их на уровень 40. В этом задании вам предлагается проанализировать результаты AB-теста, в котором создатели переместили первые ворота в Cookie Cats с 30 уровня на уровень 40. 

**Данные получены от 90 189 игроков, которые установили игру во время проведения AB-теста. Переменные:**

* `userid` - уникальный номер, идентифицирующий каждого игрока.

* `version` - был ли игрок помещен в контрольную группу (gate_30 - ворота на уровне 30) или в тестовую группу (gate_40 - ворота на уровне 40).

* `sum_gamerounds` - количество игровых раундов, сыгранных игроком в течение первой недели после установки
* `retention_1` - вернулся ли игрок в игру через 1 день после установки
* `retention_7` - вернулся ли игрок в игру через 7 дней после установки

Когда игрок устанавливал игру, ему случайным образом назначали `gate_30` или `gate_40`.

In [244]:
import pandas as pd
df = pd.read_csv('cookie_cats.csv')
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [245]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


### Вопрос 1: есть ли зависимость между возвращаемостью игроков и расположением ворот?

#### 1.1 Проверьте гипотезу для возвращения через день

H0: вероятность вернуться после первого дня одинакова для ворот на 30-ом уровне и для ворот на 40-м уровне.
H1: вероятность вернуться после первого дня НИЖЕ для ворот на 40-м уровне

In [291]:
from statsmodels.stats.proportion import proportions_ztest

In [292]:
df['ret1'] = 1
df.loc[df.retention_1==False, 'ret1'] = 0
gate_30_ret1 = df.loc[df.version == 'gate_30', 'ret1']
gate_40_ret1 = df.loc[df.version == 'gate_40', 'ret1']

In [293]:
mean_30_ret1 = gate_30_ret1.mean()
mean_40_ret1 = gate_40_ret1.mean()
var_30_ret1 = np.var(gate_30_ret1, ddof=1)
var_40_ret1 = np.var(gate_40_ret1, ddof=1)
n_30_ret1 = gate_30_ret1.shape[0]
n_40_ret1 = gate_40_ret1.shape[0]
mean_30_ret1, mean_40_ret1

(0.4481879194630872, 0.44228274967574577)

In [301]:
ztest_ret1 = (mean_30_ret1-mean_40_ret1)/(np.sqrt(var_30_ret1/n_30_ret1 + var_40_ret1/n_40_ret1))
p_value_ret1 = (1-stats.norm.cdf(ztest_ret1))
print("ztest:", ztest_ret1, "p_value:", p_value_ret1)

ztest: 1.7840774867037263 p_value: 0.037205537485036966


H0 отвергается на уровне значимости 0.05, лучше ставить ворота на 30-м уровне

2.2 Проверьте гипотезу для возвращения через семь дней

In [295]:
df['ret7'] = 1
df.loc[df.retention_7==False, 'ret7'] = 0
gate_30_ret7 = df.loc[df.version == 'gate_30', 'ret7']
gate_40_ret7 = df.loc[df.version == 'gate_40', 'ret7']

In [299]:
mean_30_ret7 = gate_30_ret7.mean()
mean_40_ret7 = gate_40_ret7.mean()
var_30_ret7 = np.var(gate_30_ret7, ddof=1)
var_40_ret7 = np.var(gate_40_ret7, ddof=1)
n_30_ret7 = gate_30_ret7.shape[0]
n_40_ret7 = gate_40_ret7.shape[0]
mean_30_ret7, mean_40_ret7

(0.19020134228187918, 0.18200004396667327)

In [300]:
ztest_ret7 = (mean_30_ret7-mean_40_ret7)/(np.sqrt(var_30_ret7/n_30_ret7 + var_40_ret7/n_40_ret7))
p_value_ret7 = (1-stats.norm.cdf(ztest_ret7))
print("ztest:", ztest_ret7, "p_value:", p_value_ret7)

ztest: 3.1640289467735405 p_value: 0.0007780065933415958


H0 отвергается на уровне значимости 0.05, лучше ставить ворота на 30-м уровне

### Вопрос 2: Отличается ли среднее количество раундов в группах gate_40 и gate_30?

In [305]:
mean_gate_30 = df[df.version == 'gate_30'].sum_gamerounds.mean()
mean_gate_40 = df[df.version == 'gate_40'].sum_gamerounds.mean()
mean_gate_30, mean_gate_40

(52.45626398210291, 51.29877552814966)

### Вопрос 3: Отличается ли медианное количество раундов в группах gate_40 и gate_30?

In [307]:
med_gate_30 = df[df.version == 'gate_30'].sum_gamerounds.median()
med_gate_40 = df[df.version == 'gate_40'].sum_gamerounds.median()
med_gate_30, med_gate_40

(17.0, 16.0)

## Задача 4: Время покемонов (3 балла)

В данной задаче вам будет необходимо проанализировать различные характеристики покемонов и сравнить их между собой

<p style="align: center;"><img align=center src="https://proprikol.ru/wp-content/uploads/2020/11/kartinki-pokemonov-2.jpg"  width=500></p>

Покемоны - это маленькие существа, которые сражаются друг с другом на соревнованиях. Все покемоны имеют разные характеристики (сила атаки, защиты и т. д.) и относятся к одному или двум так называемым классам (вода, огонь и т. д.). Профессор Оук является изобретателем Pokedex, портативного устройства, которое хранит информацию обо всех существующих покемонах. Как его ведущий специалист по данным, Вы только что получили от него запрос с просьбой осуществить аналитику данных на всех устройствах Pokedex.

### Описание данных

Профессор Оук скопировал все содержимое память одного устройства Pokedex, в результате чего получился набор данных, с которым Вы будете работать в этой задаче. В этом файле каждая строка представляет характеристики одного покемона:

* pid: Numeric - ID покемона
* HP: Numeric - Очки здоровья
* Attack: Numeric - Сила обычной атаки
* Defense: Numeric - Сила обычной защиты
* Sp. Atk: Numeric - Сила специальной атаки
* Sp. Def: Numeric - Сила специальной защиты
* Speed: Numeric - Скорость движений
* Legendary: Boolean - «True», если покемон редкий
* Class 1: Categorical - Класс покемона
* Class 2: Categorical - Класс покемона

### Вопрос 1

Профессор Оук подозревает, что покемоны в классе `grass` имеют более сильную обычную атаку, чем у покемонов в классе `rock`. Проверьте, прав ли он, и убедите его в своем выводе статистически. 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ сотворите статистическую магию

### Вопрос 2

Также профессор Оук очень хочет проверить, правда ли нет значимой разницы между силой обычной атаки и силой обычной защиты у покемонов. Помогите ему и ответьте на вопрос, подтвердив свои выводы с помощью проверки статистических гипотез

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ сотворите статистическую магию

### Вопрос 3

И последний вопрос, который мучает профессора Оука: правда ли, что средняя скорость покемонов превышает 66.6 ? Сформулируйте подходящие гипотезы и помогите профессору Оуку, ответив на вопрос.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ сотворите статистическую магию